In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline 

In [142]:
# https://www.kaggle.com/c/competitive-data-science-final-project/data

In [5]:
transactions = pd.read_csv('sales_train.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

In [6]:
print(transactions.shape)
print(items.shape)
print(item_categories.shape)
print(shops.shape)
transactions.head(3)

(2935849, 6)
(22170, 3)
(84, 2)
(60, 2)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


#### What was the maximum total revenue among all the shops in September, 2014?

In [7]:
transactions[['Day','Month','Year']]=transactions.date.str.split('.',expand=True)
del transactions['date']

In [8]:
transactions.head(3)

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Day,Month,Year
0,0,59,22154,999.0,1.0,02,01,2013
1,0,25,2552,899.0,1.0,03,01,2013
2,0,25,2552,899.0,-1.0,05,01,2013


In [15]:
temp = transactions.groupby(['Year','Month']).get_group(('2014','09'))
temp['Revenue'] = temp.item_price * temp.item_cnt_day
temp.groupby('shop_id').Revenue.sum().max()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


7982852.1999999564

#### What item category generated the highest revenue day in summer 2014?

In [128]:
temp = transactions.groupby(['Year']).get_group('2014')
temp = temp[temp['Month'].isin(['06','07','08'])]
temp['Revenue']= temp.item_price * temp.item_cnt_day
temp = temp.groupby(['item_id']).Revenue.max()
temp.head(3)

item_id
1    4490.0
2      58.0
3     100.0
Name: Revenue, dtype: float64

In [124]:
items[items['item_id']==temp.idxmax()]

,item_name,item_id,item_category_id
16790,"Одни из нас. Обновленная версия [PS4, русская ...",16790,20


#### How many items are there, such that their price stays constant (to the best of our knowledge) during the whole period of time?

In [122]:
temp = transactions.groupby('item_id').item_price.unique()
temp[temp.apply(lambda x: len(x)) == 1].count()

5926

#### What was the variance of the number of sold items per day sequence for the shop with shop_id = 25 in December, 2014?

In [141]:
temp = transactions.groupby(['shop_id', 'Month', 'Year']).get_group((25, '12', '2014'))
# Set ddof=0 if you want biased estimate.
temp.groupby('Day').item_cnt_day.sum().var(ddof=1) 

117167.70229885059